In [ ]:
# app_feedback_pipeline.py
# ------------------------------------------------------------
# End-to-end pipeline for app-store review analytics
# - Ingest + clean (Google Play + Apple)
# - ML sentiment (DistilBERT SST-2) with hybrid fallback
# - Zero-shot topic tagging (BART-MNLI), batched
# - Optional summaries per topic (BART-CNN)
# - N-gram mining & feature-request flags
# - App-level summary report
#
# Configuration is at the top; all outputs go to ./outputs/.
# Set HUGGINGFACE_TOKEN in your environment for Inference API use (optional).
# ------------------------------------------------------------

import os
import re
import math
import json
from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Transformers (local pipelines)
from transformers import pipeline
import torch

# Sklearn for n-grams
from sklearn.feature_extraction.text import CountVectorizer

# ----------------------
# CONFIG
# ----------------------
DATA_DIR = Path(".")
OUT_DIR = Path("./outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Input file names (adjust if yours differ)
PLAY_FILE = DATA_DIR / "Play Store Data.csv"
APPLE_FILE = DATA_DIR / "Apple_Store_Reviews.csv"

# Canonical outputs
COMBINED_CSV = OUT_DIR / "combined_app_reviews.csv"
CLEAN_CSV = OUT_DIR / "super_cleaned_app_reviews.csv"
ML_SENT_CSV = OUT_DIR / "ml_sentiment_reviews.csv"
HYBRID_SENT_CSV = OUT_DIR / "hybrid_sentiment_reviews.csv"
TOPIC_CSV = OUT_DIR / "topic_classified_reviews.csv"
APP_SUMMARY_CSV = OUT_DIR / "app_level_summary_report.csv"
NEG_SUMMARY_TXT = OUT_DIR / "negative_review_summaries.txt"
NGRAMS_CSV = OUT_DIR / "top_ngrams_negative_neutral.csv"
FEATURE_REQUESTS_CSV = OUT_DIR / "feature_requests.csv"

# Models
MODEL_SENTIMENT = "distilbert-base-uncased-finetuned-sst-2-english"
MODEL_ZS = "facebook/bart-large-mnli"
MODEL_SUM = "facebook/bart-large-cnn"

# Zero-shot labels (use multi_label to allow multiple topics)
ZS_LABELS = ["user interface", "performance", "bugs", "features", "pricing", "ads", "login issues", "customer support"]
ZS_MULTI_LABEL = True
ZS_SCORE_THRESH = 0.35  # keep labels scoring >= threshold when multi_label=True

# Placeholder review detection
PLACEHOLDERS = [
    "no review available", "no reviews available", "no comment",
    "n/a", "none", "", "null"
]

# Hybrid sentiment thresholds (when no usable text)
RATING_POS = 4.0
RATING_NEG = 2.0

# Chunking for summarization
MAX_REVIEWS_PER_TOPIC_FOR_SUMMARY = 50  # safety on input length
SUMMARY_MAX_LEN = 120
SUMMARY_MIN_LEN = 40

# ----------------------
# UTILS
# ----------------------
def safe_lower(s: str) -> str:
    try:
        return str(s).strip().lower()
    except Exception:
        return str(s)

def is_placeholder_text(s: str) -> bool:
    s = safe_lower(s)
    return any(p == s or p in s for p in PLACEHOLDERS)

def batch_iter(lst: List[str], batch_size: int):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size], i

def ensure_columns(df: pd.DataFrame, cols: List[str]):
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
    return df

def map_sst2_label(label: str) -> str:
    # Normalize to UPPER fixed set
    label = safe_lower(label)
    if "pos" in label:
        return "POSITIVE"
    if "neg" in label:
        return "NEGATIVE"
    return "NEUTRAL"

# ----------------------
# 1) INGEST + STANDARDIZE
# ----------------------
def ingest_and_standardize() -> pd.DataFrame:
    # Load
    g = pd.read_csv(PLAY_FILE)
    a = pd.read_csv(APPLE_FILE)

    # Standardize names
    g = g.rename(columns={
        "App": "App_Name",
        "Rating": "App_Rating",
        "Category": "Category",
        # if your Play CSV has review text, map here, else we'll inject placeholder
    })
    a = a.rename(columns={
        "App_Name": "App_Name",
        "Rating": "App_Rating",
        "Review_Text": "Review",
        "Category": "Category"
    })

    # Add review text for Google if missing
    if "Review" not in g.columns:
        g["Review"] = "No Review Available"

    # Source
    g["Source"] = "Google Play Store"
    a["Source"] = "Apple App Store"

    # Keep canonical columns if present
    common = ["App_Name", "App_Rating", "Review", "Category", "Source"]
    g = ensure_columns(g, common)[common]
    a = ensure_columns(a, common)[common]

    df = pd.concat([g, a], ignore_index=True)
    df.to_csv(COMBINED_CSV, index=False)
    return df

# ----------------------
# 2) CLEANING
# ----------------------
def clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates().copy()

    # Coerce rating
    if "App_Rating" in df.columns:
        df["App_Rating"] = pd.to_numeric(df["App_Rating"], errors="coerce")
        # per-app median then global
        df["App_Rating"] = df.groupby("App_Name")["App_Rating"].transform(lambda x: x.fillna(x.median()))
        df["App_Rating"] = df["App_Rating"].fillna(df["App_Rating"].median())

    # Fill missing text fields
    df["Review"] = df["Review"].fillna("No Review")
    df["Category"] = df["Category"].fillna("Unknown")
    df["Source"] = df["Source"].fillna("unknown")
    df["App_Name"] = df["App_Name"].fillna("unknown_app")

    # Normalize (lowercase/strip for processing); keep a display copy if needed
    for col in ["App_Name", "Review", "Category", "Source"]:
        df[col] = df[col].astype(str).str.strip()

    # Bound ratings to [1, 5]
    if "App_Rating" in df.columns:
        df = df[(df["App_Rating"] >= 1.0) & (df["App_Rating"] <= 5.0)]

    # Add placeholder flag
    df["Is_Empty_Review"] = df["Review"].apply(is_placeholder_text)

    df.to_csv(CLEAN_CSV, index=False)
    return df

# ----------------------
# 3) SENTIMENT (ML, BATCHED)
# ----------------------
def run_ml_sentiment(df: pd.DataFrame, text_col="Review", max_len=512, batch_size=32) -> pd.DataFrame:
    device = 0 if torch.cuda.is_available() else -1
    clf = pipeline("sentiment-analysis", model=MODEL_SENTIMENT, device=device)

    texts = df[text_col].astype(str).tolist()
    results = []
    for batch, i0 in tqdm(batch_iter(texts, batch_size), total=math.ceil(len(texts)/batch_size), desc="ML Sentiment"):
        # truncate to max_len (rough; tokenization occurs inside pipeline)
        batch = [t[:max_len] for t in batch]
        try:
            out = clf(batch)
        except Exception as e:
            # fallback: neutral for the whole batch on error
            out = [{"label": "NEUTRAL", "score": 0.0} for _ in batch]
        for r in out:
            results.append((map_sst2_label(r.get("label", "")), float(r.get("score", 0.0))))

    df = df.copy()
    df["Sentiment_Label"] = [r[0] for r in results]
    df["Sentiment_Confidence"] = [r[1] for r in results]
    df.to_csv(ML_SENT_CSV, index=False)
    return df

# ----------------------
# 4) HYBRID SENTIMENT (text if available, else rating)
# ----------------------
def apply_hybrid_sentiment(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # If review is placeholder/empty → fall back to rating
    def hybrid(row):
        if not row.get("Is_Empty_Review", False):
            # use ML result
            return row.get("Sentiment_Label", "NEUTRAL"), float(row.get("Sentiment_Confidence", 0.0))
        # rating fallback
        rating = float(row.get("App_Rating", 0.0)) if not pd.isna(row.get("App_Rating", np.nan)) else 0.0
        if rating >= RATING_POS:
            return "POSITIVE", 1.0
        elif rating <= RATING_NEG:
            return "NEGATIVE", 1.0
        else:
            return "NEUTRAL", 1.0

    hybrid_labels, hybrid_conf = [], []
    for _, r in df.iterrows():
        lab, conf = hybrid(r)
        hybrid_labels.append(lab)
        hybrid_conf.append(conf)

    df["Hybrid_Sentiment_Label"] = hybrid_labels
    df["Hybrid_Sentiment_Confidence"] = hybrid_conf
    df.to_csv(HYBRID_SENT_CSV, index=False)
    return df

# ----------------------
# 5) ZERO-SHOT TOPIC TAGGING (BATCHED)
# ----------------------
def zero_shot_topics(df: pd.DataFrame, text_col="Review", batch_size=16) -> pd.DataFrame:
    device = 0 if torch.cuda.is_available() else -1
    zsp = pipeline("zero-shot-classification", model=MODEL_ZS, device=device)

    # Only run on non-placeholder text to save time
    mask = ~df["Is_Empty_Review"].fillna(False)
    texts = df.loc[mask, text_col].astype(str).str.slice(0, 512).tolist()
    idxs = df.index[mask].tolist()

    topic_col = pd.Series(index=df.index, dtype=object)
    topics_multi = []

    for batch, i0 in tqdm(batch_iter(texts, batch_size), total=math.ceil(len(texts)/batch_size), desc="Zero-Shot Topics"):
        try:
            out = zsp(batch, candidate_labels=ZS_LABELS, multi_label=ZS_MULTI_LABEL)
        except Exception as e:
            # mark all as unknown on error
            if ZS_MULTI_LABEL:
                out = [{"labels": [], "scores": []} for _ in batch]
            else:
                out = [{"labels": ["unclassified"], "scores": [0.0]} for _ in batch]

        for j, res in enumerate(out):
            if ZS_MULTI_LABEL:
                # keep all labels above threshold
                kept = [lab for lab, sc in zip(res.get("labels", []), res.get("scores", [])) if sc >= ZS_SCORE_THRESH]
                topics_multi.append(kept if kept else ["unclassified"])
            else:
                topics_multi.append([res.get("labels", ["unclassified"])[0]])

    # Store best single topic and list of topics
    best_single = [ts[0] if ts else "unclassified" for ts in topics_multi]
    topic_col.loc[idxs] = best_single

    df = df.copy()
    df["Topic"] = topic_col.fillna("unclassified")
    df["Topics_All"] = None
    # Insert lists; pandas will store as object (JSON-serializable on save if needed)
    for k, ix in enumerate(idxs):
        df.at[ix, "Topics_All"] = topics_multi[k]

    df.to_csv(TOPIC_CSV, index=False)
    return df

# ----------------------
# 6) SUMMARIZATION (NEGATIVE REVIEWS PER TOPIC)
# ----------------------
def summarize_negative_by_topic(df: pd.DataFrame) -> None:
    device = 0 if torch.cuda.is_available() else -1
    summarizer = pipeline("summarization", model=MODEL_SUM, device=device)

    neg = df[df["Hybrid_Sentiment_Label"] == "NEGATIVE"].copy()
    topics = sorted(t for t in neg["Topic"].dropna().unique() if t != "unclassified")

    with open(NEG_SUMMARY_TXT, "w") as f:
        for t in topics:
            reviews = (
                neg.loc[neg["Topic"] == t, "Review"]
                .dropna()
                .astype(str)
                .tolist()
            )[:MAX_REVIEWS_PER_TOPIC_FOR_SUMMARY]

            if len(reviews) < 3:
                f.write(f"Topic: {t}\nSummary: Not enough data to summarize.\n\n")
                continue

            text = " ".join(reviews)[:2048]  # truncate safety
            try:
                s = summarizer(text, max_length=SUMMARY_MAX_LEN, min_length=SUMMARY_MIN_LEN, do_sample=False)
                summary = s[0]["summary_text"]
            except Exception:
                summary = "Summarization failed."
            f.write(f"Topic: {t}\nSummary: {summary}\n\n")

# ----------------------
# 7) N-GRAM MINING (NEGATIVE/NEUTRAL)
# ----------------------
def mine_ngrams(df: pd.DataFrame, ngram_range=(2, 3), top_k=50) -> pd.DataFrame:
    focus = df[df["Hybrid_Sentiment_Label"].isin(["NEGATIVE", "NEUTRAL"])].copy()
    if focus.empty:
        pd.DataFrame({"Feature": [], "Frequency": []}).to_csv(NGRAMS_CSV, index=False)
        return pd.DataFrame()

    vec = CountVectorizer(
        ngram_range=ngram_range,
        stop_words="english",
        max_features=top_k
    )
    X = vec.fit_transform(focus["Review"].astype(str))
    feats = vec.get_feature_names_out()
    counts = X.sum(axis=0).A1
    freq = pd.DataFrame({"Feature": feats, "Frequency": counts}).sort_values("Frequency", ascending=False)
    freq.to_csv(NGRAMS_CSV, index=False)
    return freq

# ----------------------
# 8) FEATURE-REQUEST FLAGS (REGEX)
# ----------------------
FEATURE_PATTERNS = [
    r"\bi wish\b",
    r"\bwould love\b",
    r"\bplease add\b",
    r"\bwould be great\b",
    r"\bit would be nice\b",
    r"\bplease include\b",
    r"\bhope you can\b",
    r"\bcan you add\b",
    r"\bmissing\b",
    r"\bshould have\b",
    r"\bmust have\b",
    r"\bneed(?:s)?\s+(?:feature|option|support)\b",
    r"\badd\s+(?:support|option|feature)\b",
    r"\bwould be helpful\b",
    r"\bshould include\b",
    r"\bcould use\b",
]

def flag_feature_requests(df: pd.DataFrame) -> pd.DataFrame:
    pat = re.compile("|".join(FEATURE_PATTERNS), flags=re.IGNORECASE)
    out = df.copy()
    out["Feature_Request"] = out["Review"].fillna("").astype(str).apply(lambda x: bool(pat.search(x)))
    out[out["Feature_Request"]].to_csv(FEATURE_REQUESTS_CSV, index=False)
    return out

# ----------------------
# 9) APP-LEVEL SUMMARY
# ----------------------
def build_app_summary(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["App_Name"] = d["App_Name"].astype(str).str.strip().str.lower()

    # sentiment distribution per app (normalized)
    sent = (
        d.groupby("App_Name")["Hybrid_Sentiment_Label"]
        .value_counts(normalize=True)
        .unstack(fill_value=0)
        .rename(columns={
            "POSITIVE": "Positive(%)", "NEGATIVE": "Negative(%)", "NEUTRAL": "Neutral(%)"
        })
    )

    # total reviews per app
    counts = d["App_Name"].value_counts().rename("Total_Reviews").to_frame()

    # most common topic per app (if any topics)
    if "Topic" in d.columns:
        top_topic = (
            d.groupby("App_Name")["Topic"]
            .agg(lambda x: x.value_counts().idxmax() if not x.dropna().empty else "unclassified")
            .rename("Top_Topic")
        )
    else:
        top_topic = pd.Series(dtype=object, name="Top_Topic")

    summary = sent.merge(counts, left_index=True, right_index=True, how="left")
    if not top_topic.empty:
        summary = summary.merge(top_topic, left_index=True, right_index=True, how="left")

    for col in ["Positive(%)", "Negative(%)", "Neutral(%)"]:
        if col in summary.columns:
            summary[col] = (summary[col] * 100).round(1)

    summary = summary.sort_values("Total_Reviews", ascending=False).reset_index().rename(columns={"index": "App_Name"})
    summary.to_csv(APP_SUMMARY_CSV, index=False)
    return summary

# ----------------------
# MAIN
# ----------------------
def main():
    print("📥 Ingesting & standardizing ...")
    df = ingest_and_standardize()

    print("🧹 Cleaning ...")
    df = clean(df)

    print("💬 Running ML sentiment (batched) ...")
    df = run_ml_sentiment(df)

    print("🔀 Applying hybrid sentiment fallback ...")
    df = apply_hybrid_sentiment(df)

    print("🏷️ Zero-shot topic tagging (batched) ...")
    df = zero_shot_topics(df)

    print("🧾 Summarizing negative reviews per topic ...")
    summarize_negative_by_topic(df)

    print("🧩 Mining top n-grams (neg/neutral) ...")
    _ = mine_ngrams(df)

    print("✨ Flagging feature requests ...")
    df = flag_feature_requests(df)

    print("📊 Building app-level summary ...")
    _ = build_app_summary(df)

    print("\n✅ Done! Artifacts saved in:", OUT_DIR.resolve())

if __name__ == "__main__":
    main()
